In [14]:
import math
import torch
import gpytorch
from matplotlib import pyplot as plt

# Simple GP regression

https://docs.gpytorch.ai/en/latest/examples/01_Exact_GPs/Simple_GP_Regression.html

In [15]:
"""Data generation"""
# Training data is 100 points in [0,1] inclusive regularly spaced
train_x = torch.linspace(0, 1, 100)
# True function is sin(2*pi*x) with Gaussian noise
train_y = torch.sin(train_x * (2 * math.pi)) + torch.randn(train_x.size()) * math.sqrt(0.04)
print(train_x.shape, train_y.shape)

torch.Size([100]) torch.Size([100])


In [16]:
"""Model definition"""
# We will use the simplest form of GP model, exact inference
class ExactGPModel(gpytorch.models.ExactGP):
    def __init__(self, train_x, train_y, likelihood):
        super(ExactGPModel, self).__init__(train_x, train_y, likelihood)
        self.mean_module = gpytorch.means.ConstantMean()
        self.covar_module = gpytorch.kernels.ScaleKernel(gpytorch.kernels.RBFKernel())

    def forward(self, x):
        mean_x = self.mean_module(x)
        covar_x = self.covar_module(x)
        return gpytorch.distributions.MultivariateNormal(mean_x, covar_x)

# initialize likelihood and model
likelihood = gpytorch.likelihoods.GaussianLikelihood()
model = ExactGPModel(train_x, train_y, likelihood)
# Find optimal model hyperparameters
model.train()
likelihood.train()

GaussianLikelihood(
  (noise_covar): HomoskedasticNoise(
    (raw_noise_constraint): GreaterThan(1.000E-04)
  )
)

In [17]:
training_iter = 50

# Use the adam optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=0.1)  # Includes GaussianLikelihood parameters

# "Loss" for GPs - the marginal log likelihood
mll = gpytorch.mlls.ExactMarginalLogLikelihood(likelihood, model)

for i in range(training_iter):
    # Zero gradients from previous iteration
    optimizer.zero_grad()
    # Output from model
    output = model(train_x)
    # Calc loss and backprop gradients
    loss = -mll(output, train_y)
    loss.backward()
    print('Iter %d/%d - Loss: %.3f   lengthscale: %.3f   noise: %.3f' % (
        i + 1, training_iter, loss.item(),
        model.covar_module.base_kernel.lengthscale.item(),
        model.likelihood.noise.item()
    ))
    optimizer.step()

Iter 1/50 - Loss: 0.938   lengthscale: 0.693   noise: 0.693
Iter 2/50 - Loss: 0.907   lengthscale: 0.644   noise: 0.644
Iter 3/50 - Loss: 0.873   lengthscale: 0.598   noise: 0.598
Iter 4/50 - Loss: 0.836   lengthscale: 0.555   noise: 0.554
Iter 5/50 - Loss: 0.794   lengthscale: 0.514   noise: 0.513
Iter 6/50 - Loss: 0.747   lengthscale: 0.475   noise: 0.474
Iter 7/50 - Loss: 0.697   lengthscale: 0.439   noise: 0.437
Iter 8/50 - Loss: 0.645   lengthscale: 0.405   noise: 0.402
Iter 9/50 - Loss: 0.596   lengthscale: 0.372   noise: 0.369
Iter 10/50 - Loss: 0.550   lengthscale: 0.342   noise: 0.339
Iter 11/50 - Loss: 0.509   lengthscale: 0.315   noise: 0.310
Iter 12/50 - Loss: 0.471   lengthscale: 0.291   noise: 0.284
Iter 13/50 - Loss: 0.435   lengthscale: 0.271   noise: 0.259
Iter 14/50 - Loss: 0.401   lengthscale: 0.255   noise: 0.237
Iter 15/50 - Loss: 0.368   lengthscale: 0.241   noise: 0.216
Iter 16/50 - Loss: 0.335   lengthscale: 0.231   noise: 0.196
Iter 17/50 - Loss: 0.303   length

# Multitask GP regression

https://docs.gpytorch.ai/en/latest/examples/03_Multitask_Exact_GPs/Multitask_GP_Regression.html

In [18]:
"""Data generation"""
train_x = torch.linspace(0, 1, 100)

train_y = torch.stack([
    torch.sin(train_x * (2 * math.pi)) + torch.randn(train_x.size()) * 0.2,
    torch.cos(train_x * (2 * math.pi)) + torch.randn(train_x.size()) * 0.2,
], -1)
print(train_x.shape, train_y.shape)

torch.Size([100]) torch.Size([100, 2])


In [19]:
"""Data loading"""
INPUT_TENSORS_FILEPATH = "data/input_tensors.pt"
WS_OUTPUT_TENSORS_FILEPATH = "data/WS_output_tensors.pt"

train_x = torch.stack(torch.load(INPUT_TENSORS_FILEPATH)).float()
train_y = torch.stack(torch.load(WS_OUTPUT_TENSORS_FILEPATH)).float()
index_x = train_x.multinomial(num_samples=250, replacement=True)
index_y = train_y.multinomial(num_samples=250, replacement=True)
train_x = train_x[index_x]
train_y = train_y[index_y]

#train_x = train_x[:250, :] #TODO reducing the training data size
#train_y = train_y[:250, :] #TODO reducing the training data size
print(train_x.shape, train_y.shape)

torch.Size([6233, 250, 3]) torch.Size([6233, 250, 5120])


In [20]:
NUM_TASKS = train_y.shape[1]
print(NUM_TASKS)

class MultitaskGPModel(gpytorch.models.ExactGP):
    def __init__(self, train_x, train_y, likelihood):
        super(MultitaskGPModel, self).__init__(train_x, train_y, likelihood)
        self.mean_module = gpytorch.means.MultitaskMean(
            gpytorch.means.ConstantMean(), num_tasks=NUM_TASKS
        )
        self.covar_module = gpytorch.kernels.MultitaskKernel(
            gpytorch.kernels.RBFKernel(), num_tasks=NUM_TASKS, rank=1
        )

    def forward(self, x):
        mean_x = self.mean_module(x)
        covar_x = self.covar_module(x)
        return gpytorch.distributions.MultitaskMultivariateNormal(mean_x, covar_x)


likelihood = gpytorch.likelihoods.MultitaskGaussianLikelihood(num_tasks=NUM_TASKS)
model = MultitaskGPModel(train_x, train_y, likelihood)
# Find optimal model hyperparameters
model.train()
likelihood.train()

250


MultitaskGaussianLikelihood(
  (raw_task_noises_constraint): GreaterThan(1.000E-04)
  (raw_noise_constraint): GreaterThan(1.000E-04)
)

In [21]:
# Use the adam optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=0.1)  # Includes GaussianLikelihood parameters

# "Loss" for GPs - the marginal log likelihood
mll = gpytorch.mlls.ExactMarginalLogLikelihood(likelihood, model)

for i in range(training_iter):
    optimizer.zero_grad()
    output = model(train_x)
    loss = -mll(output, train_y) 
    loss.backward()
    print('Iter %d/%d - Loss: %.3f' % (i + 1, training_iter, loss.item()))
    optimizer.step()
